## 02 frozenlake_naive.py

In [18]:
import random
import gym
import gym.spaces
from collections import namedtuple
import numpy as np
from tensorboardX import SummaryWriter

import torch
import torch.nn as nn
import torch.optim as optim

In [19]:
HIDDEN_SIZE = 128
BATCH_SIZE = 16
PERCENTILE = 70

In [3]:
# 코드를 재활용하기 위해서 gym의 ObservationWrapper를 DiscreteOneHotWrapper 클래스에 상속시킨다.
class DiscreteOneHotWrapper(gym.ObservationWrapper):
    def __init__(self, env):
        super(DiscreteOneHotWrapper, self).__init__(env)
        assert isinstance(env.observation_space, gym.spaces.Discrete)
        self.observation_space = gym.spaces.Box(0.0, 1.0, (env.observation_space.n, ), dtype=np.float32)

    def observation(self, observation):
        res = np.copy(self.observation_space.low)
        res[observation] = 1.0
        return res

In [4]:
class Net(nn.Module):
    def __init__(self, obs_size, hidden_size, n_actions):
        super(Net, self).__init__()
        self.net = nn.Sequential(
            nn.Linear(obs_size, hidden_size),
            nn.ReLU(),
            nn.Linear(hidden_size, n_actions)
        )

    def forward(self, x):
        return self.net(x)

In [5]:
Episode = namedtuple('Episode', field_names=['reward', 'steps'])
EpisodeStep = namedtuple('EpisodeStep', field_names=['observation', 'action'])

In [6]:
def iterate_batches(env, net, batch_size):
    batch = []
    episode_reward = 0.0
    episode_steps = []
    obs = env.reset()
    sm = nn.Softmax(dim=1)
    while True:
        obs_v = torch.FloatTensor([obs])
        act_probs_v = sm(net(obs_v))
        act_probs = act_probs_v.data.numpy()[0]
        action = np.random.choice(len(act_probs), p=act_probs)
        next_obs, reward, is_done, _ = env.step(action)
        episode_reward += reward
        episode_steps.append(EpisodeStep(observation=obs, action=action))
        if is_done:
            batch.append(Episode(reward=episode_reward, steps=episode_steps))
            episode_reward = 0.0
            episode_steps = []
            next_obs = env.reset()
            if len(batch) == batch_size:
                yield batch
                batch = []
        obs = next_obs

In [7]:
def filter_batch(batch, percentile):
    rewards = list(map(lambda s: s.reward, batch))
    reward_bound = np.percentile(rewards, percentile)
    reward_mean = float(np.mean(rewards))

    train_obs = []
    train_act = []
    for example in batch:
        if example.reward < reward_bound:
            continue
        train_obs.extend(map(lambda step: step.observation, example.steps))
        train_act.extend(map(lambda step: step.action, example.steps))

    train_obs_v = torch.FloatTensor(train_obs)
    train_act_v = torch.LongTensor(train_act)
    return train_obs_v, train_act_v, reward_bound, reward_mean

In [9]:
env = DiscreteOneHotWrapper(gym.make("FrozenLake-v0"))
# env = gym.wrappers.Monitor(env, directory="mon", force=True)
obs_size = env.observation_space.shape[0]
n_actions = env.action_space.n

net = Net(obs_size, HIDDEN_SIZE, n_actions)
objective = nn.CrossEntropyLoss()
optimizer = optim.Adam(params=net.parameters(), lr=0.01)
writer = SummaryWriter(comment="-frozenlake-naive")

for iter_no, batch in enumerate(iterate_batches(env, net, BATCH_SIZE)):
    obs_v, acts_v, reward_b, reward_m = filter_batch(batch, PERCENTILE)
    optimizer.zero_grad()
    action_scores_v = net(obs_v)
    loss_v = objective(action_scores_v, acts_v)
    loss_v.backward()
    optimizer.step()
    print("%d: loss=%.3f, reward_mean=%.1f, reward_bound=%.1f" % (iter_no, loss_v.item(), reward_m, reward_b))
    writer.add_scalar("loss", loss_v.item(), iter_no)
    writer.add_scalar("reward_bound", reward_b, iter_no)
    writer.add_scalar("reward_mean", reward_m, iter_no)
    if reward_m > 0.8:
        print("Solved!")
        break
    if iter_no > 3000: ########################################################
        print("Wrapper does not improve the score!!!") ########################
        break
writer.close()

0: loss=1.390, reward_mean=0.0, reward_bound=0.0
1: loss=1.356, reward_mean=0.0, reward_bound=0.0
2: loss=1.348, reward_mean=0.0, reward_bound=0.0
3: loss=1.352, reward_mean=0.0, reward_bound=0.0
4: loss=1.277, reward_mean=0.0, reward_bound=0.0
5: loss=1.288, reward_mean=0.1, reward_bound=0.0
6: loss=1.242, reward_mean=0.1, reward_bound=0.0
7: loss=1.122, reward_mean=0.0, reward_bound=0.0
8: loss=1.195, reward_mean=0.0, reward_bound=0.0
9: loss=1.045, reward_mean=0.0, reward_bound=0.0
10: loss=1.045, reward_mean=0.0, reward_bound=0.0
11: loss=1.038, reward_mean=0.0, reward_bound=0.0
12: loss=0.958, reward_mean=0.0, reward_bound=0.0
13: loss=0.958, reward_mean=0.0, reward_bound=0.0
14: loss=0.949, reward_mean=0.0, reward_bound=0.0
15: loss=0.914, reward_mean=0.0, reward_bound=0.0
16: loss=0.973, reward_mean=0.0, reward_bound=0.0
17: loss=0.921, reward_mean=0.0, reward_bound=0.0
18: loss=0.890, reward_mean=0.0, reward_bound=0.0
19: loss=0.866, reward_mean=0.0, reward_bound=0.0
20: loss=0

166: loss=1.026, reward_mean=0.0, reward_bound=0.0
167: loss=1.097, reward_mean=0.0, reward_bound=0.0
168: loss=1.168, reward_mean=0.0, reward_bound=0.0
169: loss=1.055, reward_mean=0.0, reward_bound=0.0
170: loss=1.095, reward_mean=0.0, reward_bound=0.0
171: loss=1.085, reward_mean=0.1, reward_bound=0.0
172: loss=1.087, reward_mean=0.0, reward_bound=0.0
173: loss=1.118, reward_mean=0.0, reward_bound=0.0
174: loss=1.095, reward_mean=0.0, reward_bound=0.0
175: loss=1.188, reward_mean=0.0, reward_bound=0.0
176: loss=1.130, reward_mean=0.0, reward_bound=0.0
177: loss=1.074, reward_mean=0.0, reward_bound=0.0
178: loss=1.244, reward_mean=0.0, reward_bound=0.0
179: loss=1.266, reward_mean=0.0, reward_bound=0.0
180: loss=1.161, reward_mean=0.0, reward_bound=0.0
181: loss=1.222, reward_mean=0.0, reward_bound=0.0
182: loss=1.193, reward_mean=0.0, reward_bound=0.0
183: loss=1.205, reward_mean=0.0, reward_bound=0.0
184: loss=1.276, reward_mean=0.0, reward_bound=0.0
185: loss=1.271, reward_mean=0.

334: loss=1.003, reward_mean=0.0, reward_bound=0.0
335: loss=0.944, reward_mean=0.0, reward_bound=0.0
336: loss=1.101, reward_mean=0.0, reward_bound=0.0
337: loss=0.958, reward_mean=0.0, reward_bound=0.0
338: loss=0.981, reward_mean=0.0, reward_bound=0.0
339: loss=1.021, reward_mean=0.1, reward_bound=0.0
340: loss=1.079, reward_mean=0.0, reward_bound=0.0
341: loss=1.001, reward_mean=0.0, reward_bound=0.0
342: loss=1.051, reward_mean=0.0, reward_bound=0.0
343: loss=0.933, reward_mean=0.0, reward_bound=0.0
344: loss=1.001, reward_mean=0.0, reward_bound=0.0
345: loss=1.028, reward_mean=0.0, reward_bound=0.0
346: loss=0.962, reward_mean=0.1, reward_bound=0.0
347: loss=0.930, reward_mean=0.0, reward_bound=0.0
348: loss=0.949, reward_mean=0.1, reward_bound=0.0
349: loss=1.041, reward_mean=0.0, reward_bound=0.0
350: loss=1.049, reward_mean=0.0, reward_bound=0.0
351: loss=1.034, reward_mean=0.0, reward_bound=0.0
352: loss=0.946, reward_mean=0.0, reward_bound=0.0
353: loss=0.975, reward_mean=0.

502: loss=1.115, reward_mean=0.0, reward_bound=0.0
503: loss=1.035, reward_mean=0.0, reward_bound=0.0
504: loss=0.970, reward_mean=0.1, reward_bound=0.0
505: loss=0.992, reward_mean=0.0, reward_bound=0.0
506: loss=1.087, reward_mean=0.0, reward_bound=0.0
507: loss=1.057, reward_mean=0.0, reward_bound=0.0
508: loss=1.158, reward_mean=0.0, reward_bound=0.0
509: loss=1.054, reward_mean=0.0, reward_bound=0.0
510: loss=1.117, reward_mean=0.0, reward_bound=0.0
511: loss=1.068, reward_mean=0.0, reward_bound=0.0
512: loss=1.065, reward_mean=0.0, reward_bound=0.0
513: loss=0.966, reward_mean=0.0, reward_bound=0.0
514: loss=1.134, reward_mean=0.0, reward_bound=0.0
515: loss=0.928, reward_mean=0.0, reward_bound=0.0
516: loss=1.022, reward_mean=0.0, reward_bound=0.0
517: loss=1.107, reward_mean=0.0, reward_bound=0.0
518: loss=1.050, reward_mean=0.0, reward_bound=0.0
519: loss=1.112, reward_mean=0.0, reward_bound=0.0
520: loss=1.012, reward_mean=0.0, reward_bound=0.0
521: loss=1.103, reward_mean=0.

664: loss=0.776, reward_mean=0.0, reward_bound=0.0
665: loss=0.793, reward_mean=0.0, reward_bound=0.0
666: loss=0.833, reward_mean=0.0, reward_bound=0.0
667: loss=0.868, reward_mean=0.0, reward_bound=0.0
668: loss=0.729, reward_mean=0.0, reward_bound=0.0
669: loss=0.874, reward_mean=0.0, reward_bound=0.0
670: loss=0.854, reward_mean=0.0, reward_bound=0.0
671: loss=0.915, reward_mean=0.0, reward_bound=0.0
672: loss=0.968, reward_mean=0.0, reward_bound=0.0
673: loss=0.801, reward_mean=0.0, reward_bound=0.0
674: loss=0.881, reward_mean=0.0, reward_bound=0.0
675: loss=0.827, reward_mean=0.0, reward_bound=0.0
676: loss=0.844, reward_mean=0.0, reward_bound=0.0
677: loss=0.929, reward_mean=0.0, reward_bound=0.0
678: loss=0.777, reward_mean=0.0, reward_bound=0.0
679: loss=0.994, reward_mean=0.0, reward_bound=0.0
680: loss=0.891, reward_mean=0.0, reward_bound=0.0
681: loss=0.759, reward_mean=0.0, reward_bound=0.0
682: loss=0.816, reward_mean=0.0, reward_bound=0.0
683: loss=0.854, reward_mean=0.

...

2298: loss=0.690, reward_mean=0.0, reward_bound=0.0
2299: loss=0.566, reward_mean=0.0, reward_bound=0.0
2300: loss=0.451, reward_mean=0.0, reward_bound=0.0
2301: loss=0.543, reward_mean=0.0, reward_bound=0.0
2302: loss=0.418, reward_mean=0.0, reward_bound=0.0
2303: loss=0.470, reward_mean=0.0, reward_bound=0.0
2304: loss=0.512, reward_mean=0.0, reward_bound=0.0
2305: loss=0.553, reward_mean=0.0, reward_bound=0.0
2306: loss=0.516, reward_mean=0.0, reward_bound=0.0
2307: loss=0.459, reward_mean=0.0, reward_bound=0.0
2308: loss=0.457, reward_mean=0.0, reward_bound=0.0
2309: loss=0.516, reward_mean=0.0, reward_bound=0.0
2310: loss=0.591, reward_mean=0.0, reward_bound=0.0
2311: loss=0.450, reward_mean=0.0, reward_bound=0.0
2312: loss=0.395, reward_mean=0.0, reward_bound=0.0
2313: loss=0.366, reward_mean=0.0, reward_bound=0.0
2314: loss=0.496, reward_mean=0.0, reward_bound=0.0
2315: loss=0.402, reward_mean=0.0, reward_bound=0.0
2316: loss=0.485, reward_mean=0.0, reward_bound=0.0
2317: loss=0

2466: loss=0.482, reward_mean=0.0, reward_bound=0.0
2467: loss=0.405, reward_mean=0.0, reward_bound=0.0
2468: loss=0.547, reward_mean=0.0, reward_bound=0.0
2469: loss=0.549, reward_mean=0.0, reward_bound=0.0
2470: loss=0.382, reward_mean=0.0, reward_bound=0.0
2471: loss=0.503, reward_mean=0.0, reward_bound=0.0
2472: loss=0.664, reward_mean=0.0, reward_bound=0.0
2473: loss=0.531, reward_mean=0.0, reward_bound=0.0
2474: loss=0.597, reward_mean=0.0, reward_bound=0.0
2475: loss=0.569, reward_mean=0.0, reward_bound=0.0
2476: loss=0.552, reward_mean=0.0, reward_bound=0.0
2477: loss=0.563, reward_mean=0.0, reward_bound=0.0
2478: loss=0.546, reward_mean=0.0, reward_bound=0.0
2479: loss=0.667, reward_mean=0.0, reward_bound=0.0
2480: loss=0.496, reward_mean=0.0, reward_bound=0.0
2481: loss=0.405, reward_mean=0.0, reward_bound=0.0
2482: loss=0.437, reward_mean=0.0, reward_bound=0.0
2483: loss=0.406, reward_mean=0.0, reward_bound=0.0
2484: loss=0.492, reward_mean=0.0, reward_bound=0.0
2485: loss=0

2624: loss=0.644, reward_mean=0.0, reward_bound=0.0
2625: loss=0.633, reward_mean=0.0, reward_bound=0.0
2626: loss=0.547, reward_mean=0.0, reward_bound=0.0
2627: loss=0.522, reward_mean=0.0, reward_bound=0.0
2628: loss=0.789, reward_mean=0.0, reward_bound=0.0
2629: loss=0.485, reward_mean=0.0, reward_bound=0.0
2630: loss=0.634, reward_mean=0.0, reward_bound=0.0
2631: loss=0.581, reward_mean=0.0, reward_bound=0.0
2632: loss=0.573, reward_mean=0.0, reward_bound=0.0
2633: loss=0.646, reward_mean=0.0, reward_bound=0.0
2634: loss=0.641, reward_mean=0.0, reward_bound=0.0
2635: loss=0.590, reward_mean=0.0, reward_bound=0.0
2636: loss=0.662, reward_mean=0.0, reward_bound=0.0
2637: loss=0.551, reward_mean=0.0, reward_bound=0.0
2638: loss=0.592, reward_mean=0.0, reward_bound=0.0
2639: loss=0.407, reward_mean=0.0, reward_bound=0.0
2640: loss=0.517, reward_mean=0.0, reward_bound=0.0
2641: loss=0.649, reward_mean=0.0, reward_bound=0.0
2642: loss=0.659, reward_mean=0.0, reward_bound=0.0
2643: loss=0

2788: loss=0.595, reward_mean=0.0, reward_bound=0.0
2789: loss=0.637, reward_mean=0.0, reward_bound=0.0
2790: loss=0.639, reward_mean=0.0, reward_bound=0.0
2791: loss=0.734, reward_mean=0.0, reward_bound=0.0
2792: loss=0.619, reward_mean=0.0, reward_bound=0.0
2793: loss=0.840, reward_mean=0.0, reward_bound=0.0
2794: loss=0.628, reward_mean=0.0, reward_bound=0.0
2795: loss=0.719, reward_mean=0.0, reward_bound=0.0
2796: loss=0.630, reward_mean=0.0, reward_bound=0.0
2797: loss=0.770, reward_mean=0.0, reward_bound=0.0
2798: loss=0.671, reward_mean=0.0, reward_bound=0.0
2799: loss=0.574, reward_mean=0.0, reward_bound=0.0
2800: loss=0.889, reward_mean=0.0, reward_bound=0.0
2801: loss=0.596, reward_mean=0.0, reward_bound=0.0
2802: loss=0.763, reward_mean=0.0, reward_bound=0.0
2803: loss=0.698, reward_mean=0.0, reward_bound=0.0
2804: loss=0.636, reward_mean=0.0, reward_bound=0.0
2805: loss=0.733, reward_mean=0.0, reward_bound=0.0
2806: loss=0.759, reward_mean=0.0, reward_bound=0.0
2807: loss=0

2949: loss=0.536, reward_mean=0.0, reward_bound=0.0
2950: loss=0.511, reward_mean=0.0, reward_bound=0.0
2951: loss=0.448, reward_mean=0.0, reward_bound=0.0
2952: loss=0.552, reward_mean=0.0, reward_bound=0.0
2953: loss=0.412, reward_mean=0.0, reward_bound=0.0
2954: loss=0.653, reward_mean=0.0, reward_bound=0.0
2955: loss=0.554, reward_mean=0.0, reward_bound=0.0
2956: loss=0.497, reward_mean=0.0, reward_bound=0.0
2957: loss=0.463, reward_mean=0.0, reward_bound=0.0
2958: loss=0.681, reward_mean=0.0, reward_bound=0.0
2959: loss=0.726, reward_mean=0.0, reward_bound=0.0
2960: loss=0.520, reward_mean=0.0, reward_bound=0.0
2961: loss=0.492, reward_mean=0.0, reward_bound=0.0
2962: loss=0.520, reward_mean=0.0, reward_bound=0.0
2963: loss=0.599, reward_mean=0.0, reward_bound=0.0
2964: loss=0.484, reward_mean=0.0, reward_bound=0.0
2965: loss=0.601, reward_mean=0.0, reward_bound=0.0
2966: loss=0.433, reward_mean=0.0, reward_bound=0.0
2967: loss=0.563, reward_mean=0.0, reward_bound=0.0
2968: loss=0

<img src="./images/LackofConvergence.png" />

Cartpole과 비교했을 때 FrozenLake는 환경, 에피소드, 보상이 다르다.
<img src="./images/RewardCartPole.png" />
보상 1.0을 받기 위해서는 목표지점에 도착해야만 가능할 뿐만 아니라, 이런 보상이 각 에피소드가 얼만큼 유익했는지(예: 목표지점까지 신속하거나 효율적으로 이동했는지)알려주질 않는다는 점이 차이가 난다. 오로지 성공 (1) 또는 실패(0)만 있을 뿐이다.  
엘리트 에피소드를 퍼센티지로 선택하는 방식은 Frozen Lake 문제에서는 완전히 잘못되었고, 훈련시키는데 나쁜 예만을 가져다 준다.
<img src="./images/RewardFrozenLake.png" />

## 03 frozenlake_tweaked.py

In [7]:
# BATCH_SIZE = 16
# PERCENTILE = 70

BATCH_SIZE = 100
PERCENTILE = 30
GAMMA = 0.9

교차 엔트로피로 Lake 문제를 해결하기 위해서는 코드 상으로 몇가지 수정이 필요하다.  
Cartpole에서는 배치크기를 16으로 하는 것으로 충분하였지만, Lake에서는 최소 100이상의 배치가 필요하다.  
오래걸리는 에피소드보다 적은시간이 소요되는 에피소드에 보상이 상대적으로 높아야 한다. 이는 할인(discount)개념을 써서 구현한다.  
FrozenLake는 Cartpole보다 성공적인 에피소드의 숫자가 적으므로 바로 바로 에피소드를 환경으로부터 샘플링하고 버릴 수 없고, 몇번의 반복동안 성공적인 에피소드를 가지고 있어야 한다.  
학습율을 줄여서 더 많은 훈련샘플들을 평균할 시간을 준다.  
성공적인 에피소드들이 50%이상 쌓이기 위해서는 적어도 5000번의 반복이 필요하다.

In [8]:
def filter_batch(batch, percentile):
#     rewards = list(map(lambda s: s.reward, batch))
#     reward_bound = np.percentile(rewards, percentile)
#     reward_mean = float(np.mean(rewards))
    disc_rewards = list(map(lambda s: s.reward * (GAMMA ** len(s.steps)), batch))
    ## discounted reward를 계산해준다.
    
    reward_bound = np.percentile(disc_rewards, percentile)

    train_obs = []
    train_act = []
    
    elite_batch = [] ## 엘리트 에피소드를 가지고 있다가 다음 학습 iteration으로 넘겨주는 리스트
    
#     for example in batch:
#         if example.reward < reward_bound:
#             continue
#         train_obs.extend(map(lambda step: step.observation, example.steps))
#         train_act.extend(map(lambda step: step.action, example.steps))
    
    for example, discounted_reward in zip(batch, disc_rewards):
        if discounted_reward > reward_bound:
            train_obs.extend(map(lambda step: step.observation, example.steps))
            train_act.extend(map(lambda step: step.action, example.steps))
            elite_batch.append(example)
    
#     train_obs_v = torch.FloatTensor(train_obs)
#     train_act_v = torch.LongTensor(train_act)
#     return train_obs_v, train_act_v, reward_bound, reward_mean


    return elite_batch, train_obs, train_act, reward_bound

In [13]:
random.seed(12345)
env = DiscreteOneHotWrapper(gym.make("FrozenLake-v0"))
# env = gym.wrappers.Monitor(env, directory="mon", force=True)
obs_size = env.observation_space.shape[0]
n_actions = env.action_space.n

net = Net(obs_size, HIDDEN_SIZE, n_actions)
objective = nn.CrossEntropyLoss()
optimizer = optim.Adam(params=net.parameters(), lr=0.001)
writer = SummaryWriter(comment="-frozenlake-tweaked")

full_batch = []
for iter_no, batch in enumerate(iterate_batches(env, net, BATCH_SIZE)):
    reward_mean = float(np.mean(list(map(lambda s: s.reward, batch))))
    full_batch, obs, acts, reward_bound = filter_batch(full_batch + batch, PERCENTILE)
    if not full_batch:
        continue
    obs_v = torch.FloatTensor(obs)
    acts_v = torch.LongTensor(acts)
    full_batch = full_batch[-500:]
    
    optimizer.zero_grad()
    action_scores_v = net(obs_v)
    loss_v = objective(action_scores_v, acts_v)
    loss_v.backward()
    optimizer.step()
    print("%d: loss=%.3f, reward_mean=%.3f, reward_bound=%.3f, batch=%d" % (iter_no, loss_v.item(), reward_mean, reward_bound, len(full_batch)))
    writer.add_scalar("loss", loss_v.item(), iter_no)
    writer.add_scalar("reward_mean", reward_mean, iter_no)
    writer.add_scalar("reward_bound", reward_bound, iter_no)
    if reward_mean > 0.8:
        print("Solved!")
        break

writer.close()

0: loss=1.364, reward_mean=0.010, reward_bound=0.000, batch=1
1: loss=1.374, reward_mean=0.050, reward_bound=0.000, batch=6
2: loss=1.373, reward_mean=0.010, reward_bound=0.000, batch=7
3: loss=1.371, reward_mean=0.000, reward_bound=0.000, batch=7
4: loss=1.366, reward_mean=0.020, reward_bound=0.000, batch=9
5: loss=1.364, reward_mean=0.000, reward_bound=0.000, batch=9
6: loss=1.372, reward_mean=0.030, reward_bound=0.000, batch=12
7: loss=1.376, reward_mean=0.050, reward_bound=0.000, batch=17
8: loss=1.376, reward_mean=0.010, reward_bound=0.000, batch=18
9: loss=1.377, reward_mean=0.020, reward_bound=0.000, batch=20
10: loss=1.378, reward_mean=0.020, reward_bound=0.000, batch=22
11: loss=1.381, reward_mean=0.010, reward_bound=0.000, batch=23
12: loss=1.378, reward_mean=0.030, reward_bound=0.000, batch=26
13: loss=1.376, reward_mean=0.010, reward_bound=0.000, batch=27
14: loss=1.373, reward_mean=0.010, reward_bound=0.000, batch=28
15: loss=1.372, reward_mean=0.000, reward_bound=0.000, b

128: loss=1.262, reward_mean=0.040, reward_bound=0.185, batch=229
129: loss=1.262, reward_mean=0.010, reward_bound=0.082, batch=230
130: loss=1.261, reward_mean=0.010, reward_bound=0.144, batch=231
131: loss=1.255, reward_mean=0.060, reward_bound=0.206, batch=221
132: loss=1.255, reward_mean=0.020, reward_bound=0.000, batch=223
133: loss=1.255, reward_mean=0.040, reward_bound=0.098, batch=226
134: loss=1.255, reward_mean=0.050, reward_bound=0.143, batch=228
135: loss=1.251, reward_mean=0.050, reward_bound=0.206, batch=228
136: loss=1.249, reward_mean=0.030, reward_bound=0.229, batch=206
137: loss=1.249, reward_mean=0.020, reward_bound=0.000, batch=208
138: loss=1.248, reward_mean=0.010, reward_bound=0.000, batch=209
139: loss=1.246, reward_mean=0.010, reward_bound=0.000, batch=210
140: loss=1.244, reward_mean=0.070, reward_bound=0.008, batch=217
141: loss=1.244, reward_mean=0.060, reward_bound=0.030, batch=222
142: loss=1.243, reward_mean=0.020, reward_bound=0.000, batch=224
143: loss=

254: loss=1.125, reward_mean=0.050, reward_bound=0.282, batch=220
255: loss=1.125, reward_mean=0.050, reward_bound=0.153, batch=224
256: loss=1.125, reward_mean=0.060, reward_bound=0.280, batch=227
257: loss=1.123, reward_mean=0.050, reward_bound=0.237, batch=229
258: loss=1.123, reward_mean=0.050, reward_bound=0.254, batch=229
259: loss=1.121, reward_mean=0.050, reward_bound=0.282, batch=229
260: loss=1.121, reward_mean=0.060, reward_bound=0.314, batch=221
261: loss=1.120, reward_mean=0.060, reward_bound=0.229, batch=224
262: loss=1.124, reward_mean=0.060, reward_bound=0.349, batch=214
263: loss=1.127, reward_mean=0.050, reward_bound=0.000, batch=219
264: loss=1.129, reward_mean=0.030, reward_bound=0.000, batch=222
265: loss=1.132, reward_mean=0.030, reward_bound=0.007, batch=225
266: loss=1.131, reward_mean=0.050, reward_bound=0.091, batch=227
267: loss=1.126, reward_mean=0.040, reward_bound=0.135, batch=228
268: loss=1.123, reward_mean=0.070, reward_bound=0.231, batch=229
269: loss=

380: loss=1.093, reward_mean=0.070, reward_bound=0.308, batch=229
381: loss=1.104, reward_mean=0.060, reward_bound=0.314, batch=225
382: loss=1.103, reward_mean=0.090, reward_bound=0.314, batch=226
383: loss=1.105, reward_mean=0.040, reward_bound=0.229, batch=228
384: loss=1.102, reward_mean=0.040, reward_bound=0.349, batch=222
385: loss=1.101, reward_mean=0.060, reward_bound=0.238, batch=225
386: loss=1.102, reward_mean=0.040, reward_bound=0.289, batch=227
387: loss=1.099, reward_mean=0.050, reward_bound=0.284, batch=229
388: loss=1.100, reward_mean=0.050, reward_bound=0.328, batch=230
389: loss=1.099, reward_mean=0.060, reward_bound=0.376, batch=231
390: loss=1.097, reward_mean=0.060, reward_bound=0.387, batch=222
391: loss=1.098, reward_mean=0.080, reward_bound=0.324, batch=225
392: loss=1.097, reward_mean=0.070, reward_bound=0.387, batch=225
393: loss=1.096, reward_mean=0.050, reward_bound=0.289, batch=227
394: loss=1.094, reward_mean=0.060, reward_bound=0.342, batch=229
395: loss=

506: loss=1.067, reward_mean=0.110, reward_bound=0.000, batch=181
507: loss=1.058, reward_mean=0.170, reward_bound=0.052, batch=195
508: loss=1.052, reward_mean=0.110, reward_bound=0.007, batch=206
509: loss=1.056, reward_mean=0.130, reward_bound=0.098, batch=213
510: loss=1.055, reward_mean=0.070, reward_bound=0.091, batch=219
511: loss=1.055, reward_mean=0.050, reward_bound=0.120, batch=223
512: loss=1.055, reward_mean=0.130, reward_bound=0.167, batch=225
513: loss=1.059, reward_mean=0.110, reward_bound=0.206, batch=225
514: loss=1.055, reward_mean=0.090, reward_bound=0.234, batch=227
515: loss=1.056, reward_mean=0.050, reward_bound=0.205, batch=229
516: loss=1.059, reward_mean=0.120, reward_bound=0.282, batch=225
517: loss=1.061, reward_mean=0.100, reward_bound=0.314, batch=219
518: loss=1.059, reward_mean=0.060, reward_bound=0.328, batch=223
519: loss=1.063, reward_mean=0.060, reward_bound=0.349, batch=218
520: loss=1.061, reward_mean=0.070, reward_bound=0.234, batch=222
521: loss=

...

20185: loss=0.039, reward_mean=0.490, reward_bound=0.277, batch=222
20186: loss=0.037, reward_mean=0.390, reward_bound=0.282, batch=219
20187: loss=0.038, reward_mean=0.400, reward_bound=0.265, batch=223
20188: loss=0.037, reward_mean=0.410, reward_bound=0.220, batch=226
20189: loss=0.038, reward_mean=0.490, reward_bound=0.282, batch=225
20190: loss=0.039, reward_mean=0.530, reward_bound=0.314, batch=213
20191: loss=0.037, reward_mean=0.400, reward_bound=0.178, batch=219
20192: loss=0.037, reward_mean=0.440, reward_bound=0.265, batch=223
20193: loss=0.040, reward_mean=0.430, reward_bound=0.301, batch=226
20194: loss=0.041, reward_mean=0.500, reward_bound=0.349, batch=215
20195: loss=0.039, reward_mean=0.480, reward_bound=0.260, batch=220
20196: loss=0.041, reward_mean=0.440, reward_bound=0.304, batch=224
20197: loss=0.039, reward_mean=0.450, reward_bound=0.314, batch=225
20198: loss=0.038, reward_mean=0.550, reward_bound=0.321, batch=227
20199: loss=0.038, reward_mean=0.370, reward_bou

20306: loss=0.039, reward_mean=0.360, reward_bound=0.430, batch=221
20307: loss=0.039, reward_mean=0.420, reward_bound=0.387, batch=224
20308: loss=0.038, reward_mean=0.600, reward_bound=0.426, batch=227
20309: loss=0.038, reward_mean=0.340, reward_bound=0.422, batch=229
20310: loss=0.040, reward_mean=0.380, reward_bound=0.328, batch=230
20311: loss=0.038, reward_mean=0.550, reward_bound=0.349, batch=229
20312: loss=0.038, reward_mean=0.420, reward_bound=0.430, batch=229
20313: loss=0.038, reward_mean=0.490, reward_bound=0.401, batch=230
20314: loss=0.038, reward_mean=0.510, reward_bound=0.464, batch=231
20315: loss=0.038, reward_mean=0.450, reward_bound=0.430, batch=231
20316: loss=0.039, reward_mean=0.510, reward_bound=0.478, batch=223
20317: loss=0.039, reward_mean=0.500, reward_bound=0.459, batch=226
20318: loss=0.038, reward_mean=0.460, reward_bound=0.454, batch=228
20319: loss=0.038, reward_mean=0.450, reward_bound=0.392, batch=229
20320: loss=0.038, reward_mean=0.430, reward_bou

20428: loss=0.018, reward_mean=0.380, reward_bound=0.109, batch=201
20429: loss=0.019, reward_mean=0.400, reward_bound=0.109, batch=208
20430: loss=0.017, reward_mean=0.380, reward_bound=0.122, batch=212
20431: loss=0.019, reward_mean=0.520, reward_bound=0.135, batch=213
20432: loss=0.018, reward_mean=0.460, reward_bound=0.167, batch=218
20433: loss=0.018, reward_mean=0.400, reward_bound=0.185, batch=212
20434: loss=0.018, reward_mean=0.360, reward_bound=0.206, batch=219
20435: loss=0.018, reward_mean=0.470, reward_bound=0.206, batch=219
20436: loss=0.018, reward_mean=0.430, reward_bound=0.203, batch=223
20437: loss=0.021, reward_mean=0.450, reward_bound=0.229, batch=221
20438: loss=0.023, reward_mean=0.460, reward_bound=0.254, batch=218
20439: loss=0.022, reward_mean=0.420, reward_bound=0.282, batch=214
20440: loss=0.021, reward_mean=0.440, reward_bound=0.311, batch=220
20441: loss=0.021, reward_mean=0.400, reward_bound=0.314, batch=207
20442: loss=0.020, reward_mean=0.520, reward_bou

20549: loss=0.014, reward_mean=0.510, reward_bound=0.349, batch=223
20550: loss=0.014, reward_mean=0.490, reward_bound=0.349, batch=225
20551: loss=0.015, reward_mean=0.410, reward_bound=0.387, batch=209
20552: loss=0.014, reward_mean=0.360, reward_bound=0.194, batch=216
20553: loss=0.014, reward_mean=0.380, reward_bound=0.254, batch=220
20554: loss=0.014, reward_mean=0.490, reward_bound=0.314, batch=223
20555: loss=0.014, reward_mean=0.450, reward_bound=0.229, batch=225
20556: loss=0.014, reward_mean=0.440, reward_bound=0.349, batch=225
20557: loss=0.014, reward_mean=0.430, reward_bound=0.349, batch=226
20558: loss=0.013, reward_mean=0.420, reward_bound=0.368, batch=228
20559: loss=0.014, reward_mean=0.440, reward_bound=0.387, batch=223


KeyboardInterrupt: 

## 04 frozenlake_nonslippery.py

In [15]:
import gym.wrappers
import gym.envs.toy_text.frozen_lake

In [16]:
random.seed(12345)
env = gym.envs.toy_text.frozen_lake.FrozenLakeEnv(is_slippery=False)
env = gym.wrappers.TimeLimit(env, max_episode_steps=100)
env = DiscreteOneHotWrapper(env)
# env = gym.wrappers.Monitor(env, directory="mon", force=True)
obs_size = env.observation_space.shape[0]
n_actions = env.action_space.n

net = Net(obs_size, HIDDEN_SIZE, n_actions)
objective = nn.CrossEntropyLoss()
optimizer = optim.Adam(params=net.parameters(), lr=0.001)
writer = SummaryWriter(comment="-frozenlake-nonslippery")

full_batch = []
for iter_no, batch in enumerate(iterate_batches(env, net, BATCH_SIZE)):
    reward_mean = float(np.mean(list(map(lambda s: s.reward, batch))))
    full_batch, obs, acts, reward_bound = filter_batch(full_batch + batch, PERCENTILE)
    if not full_batch:
        continue
    obs_v = torch.FloatTensor(obs)
    acts_v = torch.LongTensor(acts)
    full_batch = full_batch[-500:]

    optimizer.zero_grad()
    action_scores_v = net(obs_v)
    loss_v = objective(action_scores_v, acts_v)
    loss_v.backward()
    optimizer.step()
    print("%d: loss=%.3f, reward_mean=%.3f, reward_bound=%.3f, batch=%d" % (
        iter_no, loss_v.item(), reward_mean, reward_bound, len(full_batch)))
    writer.add_scalar("loss", loss_v.item(), iter_no)
    writer.add_scalar("reward_mean", reward_mean, iter_no)
    writer.add_scalar("reward_bound", reward_bound, iter_no)
    if reward_mean > 0.8:
        print("Solved!")
        break
writer.close()

AttributeError: 'NoneType' object has no attribute 'max_episode_steps'